In [1]:
import os
import json
from random import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

from experiments.UNetExperiment import UNetExperiment
from data_prep.HippocampusDatasetLoader import LoadHippocampusData

In [2]:
class Config:
    """
    Holds configuration parameters
    """
    def __init__(self):
        self.name = "ResidualUNet3D"
        self.root_dir = r"D:\Quantifying Hippocampus Volume for Alzheimer's Progression\EDA\out"
        self.n_epochs = 50
        self.learning_rate = 0.0006
        self.features_maps = 32
        self.batch_size = 8
        self.axial_width = 48
        self.patch_size = 64
        self.optimizer = "Adam"
        self.test_results_dir = r"D:\Quantifying Hippocampus Volume for Alzheimer's Progression\ML pipeline\out"

In [3]:
c = Config()

# Load data
print("Loading data...")

data = LoadHippocampusData(c.root_dir, x_shape=c.axial_width, y_shape=c.patch_size, z_shape=c.patch_size)

keys = range(len(data))
split = dict()

shuffle(list(keys))
train_keys = int(len(keys) * 0.70)
valid_keys = int(len(keys) * 0.85)

split['train'] = keys[: train_keys]
split['val'] = keys[train_keys: valid_keys]
split['test'] = keys[valid_keys: ]

# Set up and run experiment

exp = UNetExperiment(c, split, data)

Loading data...
Processed 252 files.


In [4]:
del data

In [5]:
train_losses, valid_losses, valid_dc = exp.run()

Epoch 1...
Mean Training Loss = 0.6906598210334778
Mean Validation Loss = 1.2167028188705444
Val loss decreased. Model updated.

Epoch 2...
Mean Training Loss = 0.2782845158468593
Mean Validation Loss = 1.1818660259246827
Val loss decreased. Model updated.

Epoch 3...
Mean Training Loss = 0.15856665034185757
Mean Validation Loss = 1.17638201713562
Val loss decreased. Model updated.

Epoch 4...
Mean Training Loss = 0.12243036925792694
Mean Validation Loss = 1.1727720260620118
Val loss decreased. Model updated.

Epoch 5...
Mean Training Loss = 0.10110905393958092
Mean Validation Loss = 1.1696857452392577
Val loss decreased. Model updated.

Epoch 6...
Mean Training Loss = 0.09461100941354578
Mean Validation Loss = 1.1692410230636596
Val loss decreased. Model updated.

Epoch 7...
Mean Training Loss = 0.08930505405772816
Mean Validation Loss = 1.1672738313674926
Val loss decreased. Model updated.

Epoch 8...
Mean Training Loss = 0.08768335628238591
Mean Validation Loss = 1.165289306640625
V

In [6]:
def plot_losses(train_losses, valid_losses, valid_dc):
    _, ax = plt.subplots(figsize=(10, 10))
    ax.set(xlabel='Epochs', ylabel='Loss&Dice scores')
    ax.plot(range(1, 51), np.array(train_losses), label='train_loss')
    # ax.plot(range(1, 51), np.array(valid_losses), label='valid_losses')
    ax.plot(range(1, 51), np.array(valid_dc), label='val_dice_scores')
    ax.plot(np.argmax(valid_dc), np.max(valid_dc), marker='o', color='k', markersize=6) 
    ax.set_xticks(range(1, 51, 2))
    ax.set_yticks(np.arange(0, 1.2, 0.1))
    ax.legend()

sns.set_theme(font_scale=1.3)
plot_losses(train_losses, valid_losses, valid_dc)
plt.savefig("mygraph.png")

In [7]:
results_json = exp.run_test()

results_json["config"] = vars(c)

with open(os.path.join(exp.out_dir, "results.json"), 'w') as out_file:
    json.dump(results_json, out_file, indent=2, separators=(',', ': '))

Testing...
Mean DSC1 = 0.8833133831167277
Mean DSC2 = 0.8648423816740174
Mean OBJ DSC = 0.9015958248420034

Testing complete.


In the Medical Segmentation Decathlon Challenge the highest reached DSC is 0.9 for class 1 and 0.89 for class 2 so we will continue using more advanced techniques such as transformers and self supervised learning to improve our results